In [1]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#importing the data
dataset_tv = pd.read_csv('2018-07-12_161320_mentions_by_media_type.csv')
dataset_radio = pd.read_csv('2018-07-12_161430_mentions_by_media_type.csv')
dataset_online_news = pd.read_csv('ONLINE-NEWS.csv')
dataset_stocks = pd.read_csv('MSFT.csv')

In [3]:
#removing excess information
dataset_radio_new = dataset_radio.iloc[0:31,2:3] 
dataset_onlinenews_new = dataset_online_news.iloc[:,2:3]
dataset_tv_new = dataset_tv.iloc[0:31,2:3]
dataset_stocks_new = dataset_stocks.iloc[0:31,1:2]

In [4]:
#concatinating the total data together
dataset_total = pd.concat((dataset_tv_new['MENTIONS'], dataset_radio_new['MENTIONS'], dataset_online_news['MENTIONS'], dataset_stocks_new['Open']), axis = 1)

In [5]:
dataset_total.head()

,MENTIONS,MENTIONS,MENTIONS,Open
0,287,336,13224,101.099998
1,386,290,12728,101.720001
2,277,566,13142,101.650002
3,281,437,9882,101.510002
4,567,620,4071,100.760000


In [8]:
#feature scaling (Normalisation as it is recommended whenever we are using sigmoid as activation function)
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_dataset_scaled = sc.fit_transform(dataset_total)

In [12]:
training_dataset_scaled


array([[0.37263158, 0.34012539, 0.95865298, 0.69016716],
       [0.58105263, 0.26802508, 0.90923583, 0.80519555],
       [0.35157895, 0.70062696, 0.95048321, 0.79220872],
       [0.36      , 0.4984326 , 0.62568497, 0.76623469],
       [0.96210526, 0.78526646, 0.04672711, 0.62708776],
       [0.12421053, 1.        , 0.02470858, 0.62708776],
       [0.20631579, 0.28840125, 0.66005779, 0.48794156],
       [0.60631579, 0.22727273, 0.79127229, 0.42115121],
       [0.60631579, 0.41222571, 0.85852346, 0.74026085],
       [0.18947368, 0.17868339, 0.86709176, 0.87198609],
       [0.20210526, 0.12068966, 0.54189499, 0.56215343],
       [1.        , 0.17241379, 0.06605559, 0.52319165],
       [0.48210526, 0.44827586, 0.14785294, 0.52319165],
       [0.27578947, 0.12539185, 0.82295507, 0.4860859 ],
       [0.13052632, 0.29310345, 0.79914317, 0.26716197],
       [0.07368421, 0.21159875, 0.70628674, 0.40816419],
       [0.02526316, 0.26645768, 0.65776626, 0.        ],
       [0.00842105, 0.48902821,

In [14]:
# Creating a data structure with 6 timesteps and 1 output
x_train = []
y_train = []
for i in range(6, 30):
    x_train.append(training_dataset_scaled[i-6:i, :])
    y_train.append(training_dataset_scaled[i, 3:4])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train[2]

array([[0.35157895, 0.70062696, 0.95048321, 0.79220872],
       [0.36      , 0.4984326 , 0.62568497, 0.76623469],
       [0.96210526, 0.78526646, 0.04672711, 0.62708776],
       [0.12421053, 1.        , 0.02470858, 0.62708776],
       [0.20631579, 0.28840125, 0.66005779, 0.48794156],
       [0.60631579, 0.22727273, 0.79127229, 0.42115121]])

In [42]:
print(x_train.shape[2])
print(x_train.shape[1])
print(x_train.shape[0])


4
6
24


## reshaping
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1]))

In [19]:
x_train[0]

array([[0.37263158, 0.34012539, 0.95865298, 0.69016716],
       [0.58105263, 0.26802508, 0.90923583, 0.80519555],
       [0.35157895, 0.70062696, 0.95048321, 0.79220872],
       [0.36      , 0.4984326 , 0.62568497, 0.76623469],
       [0.96210526, 0.78526646, 0.04672711, 0.62708776],
       [0.12421053, 1.        , 0.02470858, 0.62708776]])

In [20]:
#Building the RNN

#importing the keras library and pakages
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM

C:\Users\Shalini\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
#initialising the rnn
regressor = Sequential() 

In [43]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 20, return_sequences = True, input_shape = (x_train.shape[0],x_train.shape[1],x_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 20, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 20))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

ValueError: Input 0 is incompatible with layer lstm_15: expected ndim=3, found ndim=2

In [23]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [24]:
# Fitting the RNN to the Training set
regressor.fit(x_train, y_train, epochs = 10, batch_size = 32)

ValueError: Error when checking input: expected lstm_1_input to have shape (6, 1) but got array with shape (6, 4)